In [1]:
# EDA steps:
•	Import the moduels
•	Read data
•	Check data characters
•	Calculating statistics: mean, median, standard deviation
•   Plot the Target variable "logerror" 
•   Plot the number of transactions in each month
•	Correlation computation & heatmap
•   Look at each of the significant factor 

# Data pre-processing steps:
•	Import the Libraries
•	Check out & filled the missing values
•	Linear Regression
•	Polynomial regression
•	Lasso regression
•	Ridge Regression
•	ElasticNet Regression
•   Neural Network
•	Model Evaluation

# Explain the whole model, give a brief idea

In [2]:
# Import the moduels

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
color = sns.color_palette()


# Forceasting with decompasable model
from pylab import rcParams
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Datetime operations
import time

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import pickle
import gc
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Read data

In [5]:
home_train = pd.read_csv("/dbfs/FileStore/tables/train_2016_v2.csv", header='infer',parse_dates=["transactiondate"])
prop = pd.read_csv("/dbfs/FileStore/tables/properties_2016.csv", header='infer')
sample_submission = pd.read_csv("/dbfs/FileStore/tables/sample_submission.csv", header='infer')

In [6]:
home_train.shape
home_train.head()

Out[ 4 ]: 
 parcelid logerror transactiondate
0 11016594 0.0276 2016-01-01
1 14366692 -0.1684 2016-01-01
2 12098116 -0.0040 2016-01-01
3 12643413 0.0218 2016-01-02
4 14432541 -0.0050 2016-01-02

In [7]:
prop.shape
prop.head()

Out[ 5 ]: 
 parcelid airconditioningtypeid architecturalstyletypeid basementsqft \
0 10754147 NaN NaN NaN 
1 10759547 NaN NaN NaN 
2 10843547 NaN NaN NaN 
3 10859147 NaN NaN NaN 
4 10879947 NaN NaN NaN 

 bathroomcnt bedroomcnt buildingclasstypeid buildingqualitytypeid \
0 0.0 0.0 NaN NaN 
1 0.0 0.0 NaN NaN 
2 0.0 0.0 NaN NaN 
3 0.0 0.0 3.0 7.0 
4 0.0 0.0 4.0 NaN 

 calculatedbathnbr decktypeid ... numberofstories \
0 NaN NaN ... NaN 
1 NaN NaN ... NaN 
2 NaN NaN ... NaN 
3 NaN NaN ... 1.0 
4 NaN NaN ... NaN 

 fireplaceflag structuretaxvaluedollarcnt taxvaluedollarcnt \
0 NaN NaN 9.0 
1 NaN NaN 27516.0 
2 NaN 650756.0 1413387.0 
3 NaN 571346.0 1156834.0 
4 NaN 193796.0 433491.0 

 assessmentyear landtaxvaluedollarcnt taxamount taxdelinquencyflag \
0 2015.0 9.0 NaN NaN 
1 2015.0 27516.0 NaN NaN 
2 2015.0 762631.0 20800.37 NaN 
3 2015.0 585488.0 14557.57 NaN 
4 2015.0 239695.0 5725.17 NaN 

 taxdelinquencyyear censustractandblock 
0 NaN NaN 
1 NaN NaN 
2 NaN NaN 
3 NaN NaN 
4 NaN NaN 

[5 rows x 58 columns]

In [8]:
# Check data characters & Calculating statistics: mean, median, standard deviation

In [9]:
me = np.mean(home_train['logerror']); med = np.median(home_train['logerror']); st = home_train['logerror'].std(); 
print(home_train['logerror'].describe())

count 90275.000000
mean 0.011457
std 0.161079
min -4.605000
25% -0.025300
50% 0.006000
75% 0.039200
max 4.737000
Name: logerror, dtype: float64

In [10]:
# Plot the Target variable "logerror" 

In [11]:
x = home_train['logerror']
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True ,
                                    gridspec_kw={"height_ratios": (.15, .85)})
sns.boxplot(x, ax=ax_box)
sns.distplot(x, ax=ax_hist, bins=400, kde=False)
ax_box.set(yticks=[])
sns.despine(ax=ax_hist)
sns.despine(ax=ax_box, left=True)
plt.xlim([me-2*st, me+2*st])
plt.show()

In [12]:
# Plot the number of transactions in each month

In [13]:
home_train['transaction_month'] = home_train['transactiondate'].dt.month

cnt_srs = home_train['transaction_month'].value_counts()
plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[3])
plt.xticks(rotation='vertical')
plt.xlabel('Month of transaction', fontsize=12)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.show()

In [14]:
# Check properties' missing value 

In [15]:
missing_df = prop.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.ix[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')

ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, missing_df.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()

In [16]:
# Merge the table

In [17]:
home_train = pd.merge(home_train, prop, on='parcelid', how='left')
home_train.head()

Out[ 15 ]: 
 parcelid logerror transactiondate transaction_month \
0 11016594 0.0276 2016-01-01 1 
1 14366692 -0.1684 2016-01-01 1 
2 12098116 -0.0040 2016-01-01 1 
3 12643413 0.0218 2016-01-02 1 
4 14432541 -0.0050 2016-01-02 1 

 airconditioningtypeid architecturalstyletypeid basementsqft bathroomcnt \
0 1.0 NaN NaN 2.0 
1 NaN NaN NaN 3.5 
2 1.0 NaN NaN 3.0 
3 1.0 NaN NaN 2.0 
4 NaN NaN NaN 2.5 

 bedroomcnt buildingclasstypeid ... numberofstories \
0 3.0 NaN ... NaN 
1 4.0 NaN ... NaN 
2 2.0 NaN ... NaN 
3 2.0 NaN ... NaN 
4 4.0 NaN ... 2.0 

 fireplaceflag structuretaxvaluedollarcnt taxvaluedollarcnt \
0 NaN 122754.0 360170.0 
1 NaN 346458.0 585529.0 
2 NaN 61994.0 119906.0 
3 NaN 171518.0 244880.0 
4 NaN 169574.0 434551.0 

 assessmentyear landtaxvaluedollarcnt taxamount taxdelinquencyflag \
0 2015.0 237416.0 6735.88 NaN 
1 2015.0 239071.0 10153.02 NaN 
2 2015.0 57912.0 11484.48 NaN 
3 2015.0 73362.0 3048.74 NaN 
4 2015.0 264977.0 5488.96 NaN 

 taxdelinquencyyear censustractandblock 
0 NaN 6.037107e+13 
1 NaN NaN 
2 NaN 6.037464e+13 
3 NaN 6.037296e+13 
4 NaN 6.059042e+13 

[5 rows x 61 columns]

In [19]:
# Let us just impute the missing values with mean values to compute correlation coefficients #
median_values =home_train.median(axis=0)
home_train_new = home_train.fillna(median_values, inplace=True)

# Now let us look at the correlation coefficient of each of these variables #
x_cols = [col for col in home_train_new.columns if col not in ['logerror'] if home_train_new[col].dtype=='float64']

labels = []
values = []
for col in x_cols:
    labels.append(col)
    values.append(np.corrcoef(home_train_new[col].values, home_train_new.logerror.values)[0,1])
corr_df = pd.DataFrame({'col_labels':labels, 'corr_values':values})
corr_df = corr_df.sort_values(by='corr_values')
    
ind = np.arange(len(labels))
width = 0.9
fig, ax = plt.subplots(figsize=(12,40))
rects = ax.barh(ind, np.array(corr_df.corr_values.values), color='y')
ax.set_yticks(ind)
ax.set_yticklabels(corr_df.col_labels.values, rotation='horizontal')
ax.set_xlabel("Correlation coefficient")
ax.set_title("Correlation coefficient of the variables")
#autolabel(rects)
plt.show()

In [20]:
# Correlation

In [21]:
corr_zero_cols = ['assessmentyear', 'storytypeid', 'pooltypeid2', 'pooltypeid7', 'pooltypeid10', 'poolcnt', 'decktypeid', 'buildingclasstypeid']
for col in corr_zero_cols:
  print(col, len(home_train_new[col].unique()))

assessmentyear 1
storytypeid 1
pooltypeid2 1
pooltypeid7 1
pooltypeid10 1
poolcnt 1
decktypeid 1
buildingclasstypeid 1

In [22]:
corr_df_sel = corr_df.ix[(corr_df['corr_values']>0.02) | (corr_df['corr_values'] < -0.01)]
corr_df_sel

Out[ 19 ]: 
 col_labels corr_values
21 heatingorsystemtypeid -0.019586
45 structuretaxvaluedollarcnt 0.022264
4 bedroomcnt 0.025467
3 bathroomcnt 0.027889
18 fullbathcnt 0.028197
7 calculatedbathnbr 0.028788
10 calculatedfinishedsquarefeet 0.038488
11 finishedsquarefeet12 0.039694

In [23]:
# Correlation heatmap

In [24]:
cols_to_use = corr_df_sel.col_labels.tolist()

temp_df = home_train[cols_to_use]
corrmat = temp_df.corr(method='spearman')
f, ax = plt.subplots(figsize=(8, 8))

# Draw the heatmap using seaborn
sns.heatmap(corrmat, vmax=1., square=True)
plt.title("Important variables correlation map", fontsize=15)
plt.show()

In [25]:
cols_to_use = corr_df_sel.col_labels.tolist()

temp_df =home_train[cols_to_use]
corrmat = temp_df.corr(method='spearman')
f, ax = plt.subplots(figsize=(8, 8))

# Draw the heatmap using seaborn
sns.heatmap(corrmat, vmax=1., square=True)
plt.title("Important variables correlation map", fontsize=15)
plt.show()

In [26]:
# Look at each of the significant factor

In [27]:
# finished square feet
col = "finishedsquarefeet12"
ulimit = np.percentile(home_train[col].values, 99.5)
llimit = np.percentile(home_train[col].values, 0.5)
home_train[col].ix[home_train[col]>ulimit] = ulimit
home_train[col].ix[home_train[col]<llimit] = llimit

plt.figure(figsize=(12,12))
sns.jointplot(x=home_train.finishedsquarefeet12.values, y=home_train.logerror.values, size=10, color=color[4])
plt.ylabel('Log Error', fontsize=12)
plt.xlabel('Finished Square Feet 12', fontsize=12)
plt.title("Finished square feet 12 Vs Log error", fontsize=15)
plt.show()

In [28]:
# bathroom count
plt.figure(figsize=(12,8))
sns.countplot(x="bathroomcnt", data=home_train)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Bathroom', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of Bathroom count", fontsize=15)
plt.show()

In [29]:
# Bedroom count
plt.figure(figsize=(12,8))
sns.countplot(x="bedroomcnt", data=home_train)
plt.ylabel('Frequency', fontsize=12)
plt.xlabel('Bedroom Count', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of Bedroom count", fontsize=15)
plt.show()

In [30]:
# Tax amount
col = "taxamount"
ulimit = np.percentile(home_train[col].values, 99.5)
llimit = np.percentile(home_train[col].values, 0.5)
home_train[col].ix[home_train[col]>ulimit] = ulimit
home_train[col].ix[home_train[col]<llimit] = llimit

plt.figure(figsize=(12,12))
sns.jointplot(x=home_train['taxamount'].values, y=home_train['logerror'].values, size=10, color='g')
plt.ylabel('Log Error', fontsize=12)
plt.xlabel('Tax Amount', fontsize=12)
plt.title("Tax Amount Vs Log error", fontsize=15)
plt.show()

In [31]:
# year built
from ggplot import *
ggplot(aes(x='yearbuilt', y='logerror'), data=home_train) + \
    geom_point(color='steelblue', size=1) + \
    stat_smooth()

Out[ 28 ]: <ggplot: (8733698534271)>

In [32]:
# latitude and longitude
ggplot(aes(x='latitude', y='longitude', color='logerror'), data=home_train) + \
    geom_point() + \
    scale_color_gradient(low = 'red', high = 'blue')

Out[ 29 ]: <ggplot: (8733698118934)>

In [33]:
# Import PCA
# from sklearn.decomposition import PCA

# Create a PCA model with 2 components: pca
# pca = PCA(n_components=2)

# Fit the PCA instance to thesamples
#pca.fit(home_train)

# Transform the scaled samples: pca_features
# pca_features = pca.transform(home_train)

# Print the shape of pca_features
# print(pca_features.shape)

In [34]:
# Linear Regression
from sklearn.linear_model import LinearRegression

home_train = pd.read_csv("/dbfs/FileStore/tables/train_2016_v2.csv", header='infer',parse_dates=["transactiondate"])
prop = pd.read_csv("/dbfs/FileStore/tables/properties_2016.csv", header='infer')
sample_submission = pd.read_csv("/dbfs/FileStore/tables/sample_submission.csv", header='infer')

def get_features(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"])
    df["transactiondate_year"] = df["transactiondate"].dt.year
    df["transactiondate_month"] = df["transactiondate"].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

def EST(y, ypred):
    #logerror=log(Zestimate)−log(SalePrice)
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)  
  
train = pd.merge(home_train, prop, how='left', on='parcelid')
y = train['logerror'].values
test = pd.merge(sample_submission, prop, how='left', left_on='ParcelId', right_on='parcelid')


exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c] == 'O'] + ['logerror','parcelid']
col = [c for c in train.columns if c not in exc]

train = get_features(train[col])
test['transactiondate'] = '2016-01-01' #should use the most common training date
test = get_features(test[col])

reg = LinearRegression(n_jobs=-1)
reg.fit(train, y)

# Make predictions using the testing set
y_pred = reg.predict(train)

print(EST(y, reg.predict(train)))

0.0683708178863

In [35]:
# Polynomial regression


In [36]:
# Lasso regression
from sklearn import linear_model
from sklearn.linear_model import Lasso

home_train = pd.read_csv("/dbfs/FileStore/tables/train_2016_v2.csv", header='infer',parse_dates=["transactiondate"])
prop = pd.read_csv("/dbfs/FileStore/tables/properties_2016.csv", header='infer')
sample_submission = pd.read_csv("/dbfs/FileStore/tables/sample_submission.csv", header='infer')

def get_features(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"])
    df["transactiondate_year"] = df["transactiondate"].dt.year
    df["transactiondate_month"] = df["transactiondate"].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

def EST(y, ypred):
    #logerror=log(Zestimate)−log(SalePrice)
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)  
  
train = pd.merge(home_train, prop, how='left', on='parcelid')
y = train['logerror'].values
test = pd.merge(sample_submission, prop, how='left', left_on='ParcelId', right_on='parcelid')


exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c] == 'O'] + ['logerror','parcelid']
col = [c for c in train.columns if c not in exc]

train = get_features(train[col])
test['transactiondate'] = '2016-01-01' #should use the most common training date
test = get_features(test[col])

reg = linear_model.Lasso(alpha=0.1)
reg.fit(train, y)
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

# Make predictions using the testing set
y_pred = reg.predict(train)

print(EST(y, reg.predict(train)))

0.0683651804894

In [37]:
# Ridge Regression
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import Ridge

home_train = pd.read_csv("/dbfs/FileStore/tables/train_2016_v2.csv", header='infer',parse_dates=["transactiondate"])
prop = pd.read_csv("/dbfs/FileStore/tables/properties_2016.csv", header='infer')
sample_submission = pd.read_csv("/dbfs/FileStore/tables/sample_submission.csv", header='infer')


def get_features(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"])
    df["transactiondate_year"] = df["transactiondate"].dt.year
    df["transactiondate_month"] = df["transactiondate"].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

def EST(y, ypred):
    #logerror=log(Zestimate)−log(SalePrice)
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)  
  
train = pd.merge(home_train, prop, how='left', on='parcelid')
y = train['logerror'].values
test = pd.merge(sample_submission, prop, how='left', left_on='ParcelId', right_on='parcelid')


exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c] == 'O'] + ['logerror','parcelid']
col = [c for c in train.columns if c not in exc]

train = get_features(train[col])
test['transactiondate'] = '2016-01-01' #should use the most common training date
test = get_features(test[col])

reg = linear_model.Ridge(alpha=.5)
reg.fit(train, y) 
Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

# Make predictions using the testing set
y_pred = reg.predict(train)

print(EST(y, reg.predict(train)))

0.0683863689167

In [38]:
from sklearn.metrics import mean_squared_error
test_ = test.iloc[90274:,].values
y_pred_ = y_pred.head(90274)
mean_squared_error(y_pred_,test_)


--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-1080040030268255> in <module> () 
 1 from sklearn . metrics import mean_squared_error 
 2 test_ = test . iloc [ 90274 : , ] . values 
 ----> 3 y_pred_ = y_pred . head ( 90274 ) 
 4 mean_squared_error ( y_pred_ , test_ ) 

 AttributeError : 'numpy.ndarray' object has no attribute 'head'

In [39]:
print(test_.shape)
print(y_pred_.shape)

(90274, 55)
(90274,)

In [40]:
# ElasticNet Regression
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

home_train = pd.read_csv("/dbfs/FileStore/tables/train_2016_v2.csv", header='infer',parse_dates=["transactiondate"])
prop = pd.read_csv("/dbfs/FileStore/tables/properties_2016.csv", header='infer')
sample_submission = pd.read_csv("/dbfs/FileStore/tables/sample_submission.csv", header='infer')

def get_features(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"])
    df["transactiondate_year"] = df["transactiondate"].dt.year
    df["transactiondate_month"] = df["transactiondate"].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

def EST(y, ypred):
    #logerror=log(Zestimate)−log(SalePrice)
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)  
  
train = pd.merge(home_train, prop, how='left', on='parcelid')
y = train['logerror'].values
test = pd.merge(sample_submission, prop, how='left', left_on='ParcelId', right_on='parcelid')


exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c] == 'O'] + ['logerror','parcelid']
col = [c for c in train.columns if c not in exc]

train = get_features(train[col])
test['transactiondate'] = '2016-01-01' #should use the most common training date
test = get_features(test[col])

train, y = make_regression(n_features=2, random_state=0)
regr = ElasticNet(random_state=0)
regr.fit(train, y)
ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=0, selection='cyclic', tol=0.0001, warm_start=False)

print(y.shape)
print(train.shape)
# Make predictions using the testing set
y_pred = reg.predict(train)

print(EST(y, reg.predict(train)))

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-1066035114152897> in <module> () 
 41 print ( train . shape ) 
 42 # Make predictions using the testing set 
 ---> 43 y_pred = reg . predict ( train ) 
 44 
 45 print ( EST ( y , reg . predict ( train ) ) ) 

 /databricks/python/lib/python3.5/site-packages/sklearn/linear_model/base.py in predict (self, X) 
 266 Returns predicted values . 
 267 """
 --> 268 return self . _decision_function ( X ) 
 269 
 270 _preprocess_data = staticmethod ( _preprocess_data ) 

 /databricks/python/lib/python3.5/site-packages/sklearn/linear_model/base.py in _decision_function (self, X) 
 251 X = check_array ( X , accept_sparse = [ 'csr' , 'csc' , 'coo' ] ) 
 252 return safe_sparse_dot(X, self.coef_.T,
 --> 253 dense_output=True) + self.intercept_
 254 
 255 def predict ( self , X ) : 

 /databricks/python/lib/python3.5/site-packages/sklearn/utils/extmath.py in safe_sparse_dot (a, b, dense_output) 
 187 return ret 
 188 else : 
 --> 189 return fast_dot ( a , b ) 
 190 
 191 

 ValueError : shapes (100,2) and (55,) not aligned: 2 (dim 1) != 55 (dim 0)

In [41]:
# Neural Newtwork


In [42]:
# Model Evaluation